<a href="https://colab.research.google.com/github/NavedAFZ/hindi_vovels-consonent_recognition/blob/master/hindi_resnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
transform = transforms.Compose([
    transforms.Resize(224), 
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    ])

In [0]:
import os
from PIL import Image
import matplotlib.pyplot as plt

import torch
import torchvision
from torch.utils.data import DataLoader, Dataset, random_split
import torchvision.transforms as transforms

#For converting the dataset to torchvision dataset format
class VowelConsonantDataset(Dataset):
    def __init__(self, file_path,train=True,transform=None):
        self.transform = transform
        self.file_path=file_path
        self.train=train
        self.file_names=[file for _,_,files in os.walk(self.file_path) for file in files]
        self.len = len(self.file_names)
        if self.train:
            self.classes_mapping=self.get_classes()
    def __len__(self):
        return len(self.file_names)
    
    def __getitem__(self, index):
        file_name=self.file_names[index]
        image_data=self.pil_loader(self.file_path+"/"+file_name)
        if self.transform:
            image_data = self.transform(image_data)
        if self.train:
            file_name_splitted=file_name.split("_")
            Y1 = self.classes_mapping[file_name_splitted[0]]
            Y2 = self.classes_mapping[file_name_splitted[1]]
            z1,z2=torch.zeros(10),torch.zeros(10)
            z1[Y1-10],z2[Y2]=1,1
            label=torch.stack([z1,z2])

            return image_data, label

        else:
            return image_data, file_name
          
    def pil_loader(self,path):
        with open(path, 'rb') as f:
            img = Image.open(f)
            return img.convert('RGB')

      
    def get_classes(self):
        classes=[]
        for name in self.file_names:
            name_splitted=name.split("_")
            classes.extend([name_splitted[0],name_splitted[1]])
        classes=list(set(classes))
        classes_mapping={}
        for i,cl in enumerate(sorted(classes)):
            classes_mapping[cl]=i
        return classes_mapping
    

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm_notebook 
import torchvision
import matplotlib.pyplot as plt
from torchvision import datasets

import torchvision.transforms as transforms

import numpy as np
import pandas as pd

train_on_gpu = torch.cuda.is_available()

In [0]:
batch_size=120

In [0]:
full_data = VowelConsonantDataset("/content/gdrive/My Drive/kaggle/train/train",train=True,transform=transform)
train_size = int(0.9 * len(full_data))
test_size = len(full_data) - train_size

train_data, validation_data = random_split(full_data, [train_size, test_size])

train_loader = torch.utils.data.DataLoader(train_data, batch_size, shuffle=True)
validation_loader = torch.utils.data.DataLoader(validation_data, batch_size, shuffle=True)

In [0]:
dataiter = iter(validation_loader)
images, labels = dataiter.next()
in1=len(images)


In [9]:
print(images[0].shape)
print(images.shape)

torch.Size([3, 224, 224])
torch.Size([120, 3, 224, 224])


In [11]:
a1={}
a1=full_data.get_classes()
print(a1)

{'C0': 0, 'C1': 1, 'C2': 2, 'C3': 3, 'C4': 4, 'C5': 5, 'C6': 6, 'C7': 7, 'C8': 8, 'C9': 9, 'V0': 10, 'V1': 11, 'V2': 12, 'V3': 13, 'V4': 14, 'V5': 15, 'V6': 16, 'V7': 17, 'V8': 18, 'V9': 19}


In [10]:
resnet_vo = models.resnet18(pretrained=True)
print(resnet_vo)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/checkpoints/resnet18-5c106cde.pth



ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
 

In [11]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [12]:
import torch.nn as nn
from torchvision import models
resnet_vo = models.resnet18(pretrained=True).to(device)
for param in resnet_vo.parameters():
    param.requires_grad = False
num_classes=10    
in_features = resnet_vo.fc.in_features
resnet_vo.fc = nn.Linear(in_features, num_classes)    
for param in resnet_vo.parameters():
    if param.requires_grad:
        print(param.shape)
resnet_vo = resnet_vo.to(device)
loss_fn = nn.CrossEntropyLoss()
opt = optim.SGD(resnet_vo.parameters(), lr=0.01)   

torch.Size([10, 512])
torch.Size([10])


In [0]:
from tqdm import tqdm_notebook 


In [0]:
# for vovels
def final_evaluation_vo(data_loader):
  total, correct = 0, 0
  #total,correct=total.to(device),correct.to(device)
  for data in validation_loader:
          inputs, labels = data 
          inputs, labels=inputs.to(device), labels.to(device)
          print(labels.shape)            #labels=N*2*10
                                            #input=N*3*64*64

          
          outputs = resnet_vo(inputs)    
          #outputs=outputs.to(device)   #(N*10)
          _,la1=torch.max(outputs.data, 1)
          #la1=la1.to(device)
          #print(outputs.shape,la1.shape)    #(N)

          """  0 :vovels
               1: consonent  """                                   
          a1=divide(labels,0)
          a1=a1.to(device)                #(N*10)         
          #print(a1.shape)
          _,pred=torch.max(a1.data, 1) 
          pred=pred.to(device)
          #print(a1.shape,pred.shape)      #(N)
          #print(pred.shape)
          

          total += labels.size(0)
          correct += (pred == la1).sum().item()
  return 100 * correct / total

In [0]:
def divide(labels,m):
  A=labels.shape[0]
  B=labels.shape[1]
  C=labels.shape[2]
  a1=torch.zeros(A,C)
  a2=torch.zeros(A,C)
  for i in range(A):
    for j in range(B):
      for k in range(C):
        if j==0:
          a1[i][k]=labels[i][0][k]
        elif j==1:
          a2[i][k]=labels[i][1][k]
  if m==0:
    return a1
  elif m==1:
    return a2  

In [57]:
from tqdm import tqdm_notebook 
#resnet_vo.load_state_dict(best_model)

resnet_vo.load_state_dict(torch.load( "/content/gdrive/My Drive/kaggle/m1.ipnyb"))

<All keys matched successfully>

In [0]:
%%time
loss_arr = []
loss_epoch_arr = []
epochs = 100
for num_epoch in tqdm_notebook(range(epochs), total=epochs, unit="epoch"):
#for epoch in range(max_epochs):

    for i, data in enumerate(train_loader, 0):

        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        opt.zero_grad()

        outputs1 = resnet_vo(inputs)       #(netvovel::for vovels)
        outputs1 =outputs1.to(device) 
        
        a1=divide(labels,0)         #for vovels   shape:N*10
        _,pred1=torch.max(a1.data, 1)
        a1,pred1=a1.to(device),pred1.to(device)
        print(a1.shape)             #shape:N
        
                   #shape:N
        loss1 = loss_fn(outputs1, pred1)   #for vovels 
        loss1.backward()
        opt.step()
        
        
        loss_arr.append(loss1.item())
        
    loss_epoch_arr.append(loss1.item())
        
    print('Epoch: %d/%d, Test acc: %0.2f, Train acc: %0.2f' % (num_epoch, epochs, final_evaluation_vo(validation_loader), final_evaluation_vo(train_loader)))
    
    
plt.plot(loss_epoch_arr)
plt.show()
#for num_epoch in tqdm_notebook(range(epochs), total=epochs, unit="epoch"):
#from tqdm import tqdm_notebook 

"""loss_epoch_arr = []
max_epochs = 4

min_loss = 1000

n_iters = np.ceil(50000/batch_size)

for epoch in range(max_epochs):

    for i, data in enumerate(trainloader, 0):

        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        opt.zero_grad()

        outputs = resnet(inputs)
        loss = loss_fn(outputs, labels)
        loss.backward()
        opt.step()
        
        if min_loss > loss.item():
            min_loss = loss.item()
            best_model = copy.deepcopy(resnet.state_dict())
            print('Min loss %0.2f' % min_loss)
        
        if i % 100 == 0:
            print('Iteration: %d/%d, Loss: %0.2f' % (i, n_iters, loss.item()))
            
        del inputs, labels, outputs
        torch.cuda.empty_cache()
        
    loss_epoch_arr.append(loss.item())
        
    print('Epoch: %d/%d, Test acc: %0.2f, Train acc: %0.2f' % (
        epoch, max_epochs, 
        evaluation(testloader, resnet), evaluation(trainloader, resnet)))
    
    
plt.plot(loss_epoch_arr)
plt.show()"""

In [0]:
torch.save(resnet_vo.state_dict(), "/content/gdrive/My Drive/kaggle/m1.ipnyb")

In [37]:
import copy

#print(final_evaluation(trainloader, vgg), evaluation(testloader, vgg))
best_model = copy.deepcopy(resnet_vo.state_dict())
resnet_vo.load_state_dict(best_model)
print(final_evaluation_vo(train_loader), final_evaluation_vo(validation_loader))


torch.Size([120, 2, 10])
torch.Size([120, 2, 10])
torch.Size([120, 2, 10])
torch.Size([120, 2, 10])
torch.Size([120, 2, 10])
torch.Size([120, 2, 10])
torch.Size([120, 2, 10])
torch.Size([120, 2, 10])
torch.Size([40, 2, 10])
torch.Size([120, 2, 10])
torch.Size([120, 2, 10])
torch.Size([120, 2, 10])
torch.Size([120, 2, 10])
torch.Size([120, 2, 10])
torch.Size([120, 2, 10])
torch.Size([120, 2, 10])
torch.Size([120, 2, 10])
torch.Size([40, 2, 10])
39.7 38.7
